In [1]:
from nwm_reanalysis import NWM
from dask.distributed import Client, LocalCluster
# import asyncio
import datetime

In [2]:
start_date = "2000-01-01"
end_date = "2010-12-31"
# comids = [2043493, 20873280, 7590453]
# comids = [2043493]
# comids = [10327875]   # waterbody
comids = [9234683]

nwm = NWM(start_date=start_date, end_date=end_date, comids=comids, waterbody=False)
scheduler = LocalCluster(n_workers=10, threads_per_worker=2, processes=False)
scheduler

LocalCluster(a28e0804, 'inproc://204.46.77.97/23372/1', workers=10, threads=20, memory=127.76 GiB)

In [3]:
%%time
# %%prun
nwm.request_timeseries(scheduler=scheduler)
nwm.data
# 10y, 1comid: 14min CPU, 19min
# 10y, 2comid: 21min CPU, 30min
# 10y, 3comid: 32min CPU, 46min

CPU times: total: 11min 27s
Wall time: 16min 4s


<xarray.Dataset>
Dimensions:     (time: 96456, feature_id: 1)
Coordinates:
    elevation   (feature_id) float32 355.1
  * feature_id  (feature_id) int32 9234683
    gage_id     (feature_id) |S15 b'               '
    latitude    (feature_id) float32 36.0
    longitude   (feature_id) float32 -80.94
    order       (feature_id) int32 1
  * time        (time) datetime64[ns] 2000-01-01 ... 2011-01-01T23:00:00
Data variables:
    streamflow  (time, feature_id) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    velocity    (time, feature_id) float64 0.04 0.04 0.04 ... 0.05 0.05 0.05
Attributes:
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    proj4:                +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1...

In [3]:
%%time
# %%prun
nwm.request_timeseries_parallel(scheduler=scheduler)
nwm.data
# 2 core: 19min
# 4 core: 12min 18s
# 8 core: 10min 5s
# 12 core: 9min 2s


CPU times: total: 1min 30s
Wall time: 9min 42s


<xarray.Dataset>
Dimensions:     (time: 96456, feature_id: 1)
Coordinates:
  * time        (time) datetime64[ns] 2000-01-01 ... 2011-01-01T23:00:00
    elevation   (feature_id) float32 355.1
  * feature_id  (feature_id) int32 9234683
    gage_id     (feature_id) |S15 b'               '
    latitude    (feature_id) float32 36.0
    longitude   (feature_id) float32 -80.94
    order       (feature_id) int32 1
Data variables:
    streamflow  (time, feature_id) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    velocity    (time, feature_id) float64 0.04 0.04 0.04 ... 0.05 0.05 0.05
Attributes:
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    proj4:                +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1...

In [ ]:
%%time
df = nwm.set_output(return_dataframe=False)
nwm.output

In [ ]:
import s3fs
import xarray as xr
import dask

_start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
_end_date = datetime.datetime.strptime(end_date, "%Y-%m-%d") + datetime.timedelta(days=1)
request_variables = ["inflow", "outflow", "water_sfc_elev"]
request_url = nwm_21_wb_url = "s3://noaa-nwm-retrospective-2-1-zarr-pds/lakeout.zarr"

s3 = s3fs.S3FileSystem(anon=True)
store = s3fs.S3Map(root=request_url, s3=s3, check=False)

ds = xr.open_zarr(store=store, consolidated=True)
ds

In [ ]:
comids = [10327875]

In [ ]:
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ds_data = ds[request_variables].sel(feature_id=comids).sel(time=slice(
        f"{_start_date.year}-{_start_date.month}-{_start_date.day}",
        f"{_end_date.year}-{_end_date.month}-{_end_date.day}"
    )).load(optimize_graph=True, traverse=False)
ds_data

In [ ]:
%%time
nwm.request_timeseries(scheduler=scheduler, optimize=False)
nwm.data

In [ ]:
%%time
nwm.set_output()
nwm.output


-----------------------------------

In [ ]:
import xarray as xr
import fsspec
import dask
import s3fs
from datetime import datetime
from rechunker import rechunk

from dask.distributed import Client, LocalCluster

In [ ]:
n_workers = 10
scheduler = LocalCluster(n_workers=n_workers, threads_per_worker=2, processes=False)
client = Client(scheduler)
client.dashboard_link

In [ ]:
# nwm_url = 's3://noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr'
nwm_url = 's3://noaa-nwm-retrospective-2-1-zarr-pds/lakeout.zarr'
start_date = datetime.fromisoformat("2000-01-01")
end_date = datetime.fromisoformat("2010-12-31")

# STREAMFLOW 
# comids = [2043493, 20873280, 7590453]
# comids = [2043493]
# variables = ["streamflow", "velocity"]

# LAKE
comids = [10327875]
# comids = [717696]
variables = ["inflow", "outflow", "water_sfc_elev"]

start_string = f"{start_date.year}-{start_date.month}-{start_date.day}"
end_string = f"{end_date.year}-{end_date.month}-{end_date.day}"

In [ ]:
%%time
# drop_variables = ["elevation", "order", "qBtmVertRunoff", "qBucket", "qSfcLatRunoff", "q_lateral"]
ds = xr.open_zarr(fsspec.get_mapper(nwm_url, anon=True), consolidated=True)
ds

In [ ]:
%%time
# Option one, default with comid selection and time slice happening at the same time with dask
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ds_data = ds[variables].sel(feature_id=comids).sel(time=slice(start_string, end_string)).load(optimize_graph=False, traverse=False)
data = ds_data
print(f"Start: {start_string}, End: {end_string}")
data = data.to_dataframe()
data

In [ ]:
# calculate volume using surface area from nhdplus dataset and the lakeparm.nc file
import sqlite3

catchments_path = "D:\\git\\hms_kube\\hms\\Web.Services\\App_Data\\catchments.sqlite"
lakeparm_path = "D:\\data\\hms\\LAKEPARM_CONUS.nwm.v2.1.nc"

# Retrieve area from db
# con = sqlite3.connect(catchments_path)
# cur = con.cursor()
# surfacearea = cur.execute(f"SELECT AREASQKM FROM Waterbodies WHERE COMID={comids[0]}").fetchall()[0][0]
# con.close()

from netCDF4 import Dataset
import xarray as xr
import numpy as np
# lakenc = Dataset(lakeparm_path)
lakenc = xr.open_dataset(lakeparm_path)
lakenc

In [ ]:
# Convert xarray dataset to a pandas dataframe and select row where lake_id == comid

lake_df = lakenc[["lake_id", "LkArea", "OrificeE"]].to_dataframe()
lake_data = lake_df.loc[lake_df['lake_id'].isin(comids)]
lake_data

In [ ]:
# Calculate volume: (LkArea*1000000) * (water_surface_elevation - OrificeE)
lake_area = float(lake_data["LkArea"])
orifice_e = float(lake_data["OrificeE"])
print(f"Lake Area: {lake_area}, OrificeE: {orifice_e}")

In [ ]:
data["volume"] = (lake_area*1000000) * (data["water_sfc_elev"] - orifice_e)
data

In [ ]:
%%time
ds = ds.sel(time=slice(start_string, end_string))
ds_features_0 = (ds.where(ds.feature_id.isin(comids), drop=True))
ds_features = ds_features_0.copy()
ds_features = ds_features.drop('crs')

# Option two, rechunk the data
dim_chunk_sizes = {'feature_id': len(comids), 'time': len(ds_features.time)}
ds_features = ds_features.chunk(chunks=dim_chunk_sizes)
chunk_plan = {}
for vv in ds_features.variables:
    if vv in ['streamflow', 'velocity']:
        chunk_plan[vv] = tuple((dim_chunk_sizes[tt] for tt in ds_features[vv].dims))
    else: 
        chunk_plan[vv] = ds_features[vv].shape
    ds_features[vv].encoding['chunks'] = None

max_mem = f"{format(0.9 * 56 / n_workers, '.2f')}GB"
ds_features = ds_features.chunk(chunks=dim_chunk_sizes)
intermediate = "intermediate.zarr"
target = "target.zarr"

rechunked = rechunk(ds_features, target_chunks=chunk_plan, target_store=target,
                    max_mem=max_mem,
                    temp_store=intermediate)
result = rechunked.execute(retries=10)
ds = xr.open_zarr(target)
ds_data = ds[variables].load(optimize_graph=True, traverse=False)
data = ds_data
print(f"Start: {start_string}, End: {end_string}")
data

In [ ]:
%%time
# Option three, no dask slicing
ds_data = ds[variables].sel(feature_id=comids).sel(time=slice(start_string, end_string)).load(optimize_graph=False, traverse=False)
data = ds_data
print(f"Start: {start_string}, End: {end_string}")
data

In [ ]:
%%time
data = data.sel(time=slice(start_string, end_string))
data

In [ ]:
%%time
# zarr_file = nwm_url
# s3 = s3fs.S3FileSystem(anon=True)
# store = s3fs.S3Map(root=zarr_file, s3=s3, check=False)
# ds_nwm_chrtout = xr.open_zarr(store=store, consolidated=True)
# ds_nwm_chrtout

In [ ]:
%%time
ds_nwm_feature = ds_nwm_chrtout.isel(feature_id==f'{self.comids}'.encode())
streamflow_nwm = ds_nwm_feature.streamflow.load()
streamflow_nwm_df = streamflow_nwm.squeeze('feature_id').to_dataframe()
streamflow_nwm_df

In [ ]:
%%time
variables = ["streamflow", "velocity"]
start_string = f"{nwm.start_date.year}-{nwm.start_date.month}-{nwm.start_date.day}"
end_string = f"{nwm.end_date.year}-{nwm.end_date.month}-{nwm.end_date.day}"
# with dask.config.set(**{'array.slicing.split_large_chunks': True}):
ds_data = ds[variables].sel(feature_id=nwm.comids).sel(time=slice(start_string, end_string)).load(optimize_graph=False, traverse=False)
data = ds_data
print(f"Start: {start_string}, End: {end_string}")
data

In [ ]:
%%time
start_string = f"{nwm.start_date.year}-{nwm.start_date.month}-{nwm.start_date.day}"
end_string = f"{nwm.end_date.year}-{nwm.end_date.month}-{nwm.end_date.day}"
ds_data = ds.sel(feature_id=nwm.comids)
ds_data = ds_data.sel(time=slice(start_string, end_string)).load(optimize_graph=True, traverse=False).load()
ds_data

In [ ]:
ds_data = ds_data.sel(time=slice(start_string, end_string)).load(optimize_graph=True, traverse=False).load()
print(f"Start: {start_string}, End: {end_string}")
ds_data

In [ ]:
all_variables = ["elevation", "order", "qBtmVertRunoff", "qBucket", "qSfcLatRunoff", "q_lateral", "streamflow", "velocity"]
d_v = list(set(all_variables) - set(variables))
d_v